In [28]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import re

import xlrd

In [2]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
def df_describer(df):
    '''
    Функция для описания таблицы. Выдает:
    - общую информацию о таблице;
    - количество пропусков;
    - количество дубликатов;
    - описание данных.
    Принимает в качестве аргумента переменную с датафреймом.
    '''
    print('Общая информация о таблице: ')
    print(df.info())
    print('=============')
    print('Пропуски в данных: ')
    display(
        df.isna().mean().to_frame('Nan')
        .join(df.isna().mean().to_frame('Nan, %'))
        .style.format({'Nan, %': '{:,.1%}'.format})
    )
    print('=============')
    print('Количество дублей: ')
    print(df.duplicated().sum())
    print('=============')
    print('Описание данных')
    display(df.describe(include='all'))

In [4]:
df_describer(df)

Общая информация о таблице: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None
Пропуски в данных: 


,Nan,"Nan, %"
InvoiceNo,0.000000,0.0%
StockCode,0.000000,0.0%
Description,0.002683,0.3%
Quantity,0.000000,0.0%
InvoiceDate,0.000000,0.0%
UnitPrice,0.000000,0.0%
CustomerID,0.249267,24.9%
Country,0.000000,0.0%


Количество дублей: 
5268
Описание данных


<ipython-input-3-ebc6a831f7d4>:24: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display(df.describe(include='all'))


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909.0,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900.0,4070,4223,NaN,23260,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom
freq,1114.0,2313,2369,NaN,1114,NaN,NaN,495478
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,9.552250,NaN,4.611114,15287.690570,NaN
std,NaN,NaN,NaN,218.081158,NaN,96.759853,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,13953.000000,NaN


Что поищем:
1) Исследуем и уберем дубли
2) посмотрим, что за клиенты без ID - что с ними делать, их много, столько удалять нельзя
3) Что за товары без описания и что с ними делать (их немного, можно удалить, если что)
4) переведем InvoiceDate в datetime, сделаем новый столбец с датой без времени, сделаем столбец с месяцем
- кластеризация товаров
5) исследуем цены:
    5.1) Построим гистограмму
    5.2) Выявим что за товары с отрицательной стоимостью (поймем, что с ними делать)
    5.3) товары с отрицательным количеством тоже интересно
    5.4) Построим топ 10 с самыми дорогими товарами, с самыми покупаемыми товарами
    5.5) создадим столбец с total_price = Quantity*UnitPrice
6) исследуем покупателей:
    6.1) соберем таблицу с агрегацией по инвойсу, дате, месяц, страной, id-покупателя, кол-во позицией в инвойсе, кол-вом товаров, суммой
    6.1) Топ 10 стран по обороту
    6.2) Топ 10 стран по среднему чеку - ? подумать, как посчитать

Еще надо оценить доли, занимаемые странами

Построить график выручки по месяцам



так как дублей у нас менее 1%, удаляем их:

In [5]:
df = df.drop_duplicates()
df_describer(df)

Общая информация о таблице: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536641 non-null  object        
 1   StockCode    536641 non-null  object        
 2   Description  535187 non-null  object        
 3   Quantity     536641 non-null  int64         
 4   InvoiceDate  536641 non-null  datetime64[ns]
 5   UnitPrice    536641 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      536641 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.8+ MB
None
Пропуски в данных: 


,Nan,"Nan, %"
InvoiceNo,0.000000,0.0%
StockCode,0.000000,0.0%
Description,0.002709,0.3%
Quantity,0.000000,0.0%
InvoiceDate,0.000000,0.0%
UnitPrice,0.000000,0.0%
CustomerID,0.251634,25.2%
Country,0.000000,0.0%


Количество дублей: 
0
Описание данных


<ipython-input-3-ebc6a831f7d4>:24: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display(df.describe(include='all'))


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,536641.0,536641,535187,536641.000000,536641,536641.000000,401604.000000,536641
unique,25900.0,4070,4223,NaN,23260,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom
freq,1114.0,2301,2357,NaN,1114,NaN,NaN,490300
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,9.620029,NaN,4.632656,15281.160818,NaN
std,NaN,NaN,NaN,219.130156,NaN,97.233118,1714.006089,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,13939.000000,NaN


Переведем столбец с датой и временем в дискретный вид

In [6]:
df['invoce_day'] = df['InvoiceDate'].dt.date
df.info()

<ipython-input-6-1a2fdc779699>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invoce_day'] = df['InvoiceDate'].dt.date


<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536641 non-null  object        
 1   StockCode    536641 non-null  object        
 2   Description  535187 non-null  object        
 3   Quantity     536641 non-null  int64         
 4   InvoiceDate  536641 non-null  datetime64[ns]
 5   UnitPrice    536641 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      536641 non-null  object        
 8   invoce_day   536641 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 40.9+ MB


In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


Добавим столбец с общей стоимостью товара в инвойсе:

In [8]:
df['total_amt'] = df['Quantity']*df['UnitPrice']
df.sample(10)

<ipython-input-8-5750ef5c26c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_amt'] = df['Quantity']*df['UnitPrice']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
441612,574590,23318,BOX OF 6 MINI VINTAGE CRACKERS,6,2011-11-06 10:44:00,2.49,18077.0,United Kingdom,2011-11-06,14.94
117677,546397,47590A,BLUE HAPPY BIRTHDAY BUNTING,3,2011-03-11 14:37:00,5.45,12971.0,United Kingdom,2011-03-11,16.35
425408,C573321,23014,GLASS APOTHECARY BOTTLE ELIXIR,-1,2011-10-30 10:35:00,3.95,14546.0,United Kingdom,2011-10-30,-3.95
193745,553542,48185,DOORMAT FAIRY CAKE,2,2011-05-17 15:34:00,14.13,NaN,United Kingdom,2011-05-17,28.26
374946,569414,21481,FAWN BLUE HOT WATER BOTTLE,3,2011-10-04 10:11:00,8.29,NaN,United Kingdom,2011-10-04,24.87
496556,578344,16238,PARTY TIME PENCIL ERASERS,1,2011-11-24 09:21:00,0.42,NaN,United Kingdom,2011-11-24,0.42
347534,567307,20712,JUMBO BAG WOODLAND ANIMALS,10,2011-09-19 13:35:00,2.08,15078.0,United Kingdom,2011-09-19,20.80
170622,551339,21929,JUMBO BAG PINK VINTAGE PAISLEY,2,2011-04-27 17:18:00,4.13,NaN,United Kingdom,2011-04-27,8.26
347393,567303,23234,BISCUIT TIN VINTAGE CHRISTMAS,12,2011-09-19 13:20:00,2.89,17613.0,United Kingdom,2011-09-19,34.68
156764,550164,20727,LUNCH BAG BLACK SKULL.,10,2011-04-14 16:31:00,1.65,13590.0,United Kingdom,2011-04-14,16.50


Теперь посмотрим, что делать с клиентами без ID

In [9]:
df_describer(df[df['CustomerID'].isna()])

Общая информация о таблице: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 135037 entries, 622 to 541540
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    135037 non-null  object        
 1   StockCode    135037 non-null  object        
 2   Description  133583 non-null  object        
 3   Quantity     135037 non-null  int64         
 4   InvoiceDate  135037 non-null  datetime64[ns]
 5   UnitPrice    135037 non-null  float64       
 6   CustomerID   0 non-null       float64       
 7   Country      135037 non-null  object        
 8   invoce_day   135037 non-null  object        
 9   total_amt    135037 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 11.3+ MB
None
Пропуски в данных: 


,Nan,"Nan, %"
InvoiceNo,0.000000,0.0%
StockCode,0.000000,0.0%
Description,0.010767,1.1%
Quantity,0.000000,0.0%
InvoiceDate,0.000000,0.0%
UnitPrice,0.000000,0.0%
CustomerID,1.000000,100.0%
Country,0.000000,0.0%
invoce_day,0.000000,0.0%
total_amt,0.000000,0.0%


Количество дублей: 
0
Описание данных


<ipython-input-3-ebc6a831f7d4>:24: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display(df.describe(include='all'))


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
count,135037.0,135037,133583,135037.000000,135037,135037.000000,0.0,135037,135037,135037.000000
unique,3710.0,3810,3554,NaN,3161,NaN,NaN,9,277,NaN
top,573585.0,DOT,DOTCOM POSTAGE,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom,2011-08-30,NaN
freq,1114.0,694,693,NaN,1114,NaN,NaN,133572,2829,NaN
first,NaN,NaN,NaN,NaN,2010-12-01 11:52:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 10:26:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.996868,NaN,8.078342,NaN,NaN,NaN,10.719192
std,NaN,NaN,NaN,66.705155,NaN,151.924958,NaN,NaN,NaN,157.992412
min,NaN,NaN,NaN,-9600.000000,NaN,-11062.060000,NaN,NaN,NaN,-17836.460000
25%,NaN,NaN,NaN,1.000000,NaN,1.630000,NaN,NaN,NaN,2.460000


Кажется, инвойс 573585 самый крупный как в общей таблице, так и среди клиентов без ID. Посмотрим, что с ним, возможно, не так:

In [10]:
df[(df['CustomerID'].isna())&(df['InvoiceNo']==573585)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
430237,573585,11001,ASSTD DESIGN RACING CAR PEN,2,2011-10-31 14:41:00,3.29,NaN,United Kingdom,2011-10-31,6.58
430238,573585,15036,ASSORTED COLOURS SILK FAN,15,2011-10-31 14:41:00,1.63,NaN,United Kingdom,2011-10-31,24.45
430239,573585,15039,SANDALWOOD FAN,15,2011-10-31 14:41:00,1.63,NaN,United Kingdom,2011-10-31,24.45
430240,573585,15056BL,EDWARDIAN PARASOL BLACK,1,2011-10-31 14:41:00,12.46,NaN,United Kingdom,2011-10-31,12.46
430241,573585,15056N,EDWARDIAN PARASOL NATURAL,1,2011-10-31 14:41:00,12.46,NaN,United Kingdom,2011-10-31,12.46
...,...,...,...,...,...,...,...,...,...,...
431346,573585,90209C,PINK ENAMEL+GLASS HAIR COMB,3,2011-10-31 14:41:00,2.07,NaN,United Kingdom,2011-10-31,6.21
431347,573585,90214S,"LETTER ""S"" BLING KEY RING",1,2011-10-31 14:41:00,0.83,NaN,United Kingdom,2011-10-31,0.83
431348,573585,DOT,DOTCOM POSTAGE,1,2011-10-31 14:41:00,2019.05,NaN,United Kingdom,2011-10-31,2019.05
431349,573585,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2011-10-31 14:41:00,16.67,NaN,United Kingdom,2011-10-31,16.67


In [11]:
df[(df['CustomerID'].isna())&(df['InvoiceNo'].str.contains('[a-zA-Z]', regex=True))]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
11502,C537251,22429,ENAMEL MEASURING JUG CREAM,-2,2010-12-06 10:45:00,4.25,NaN,United Kingdom,2010-12-06,-8.50
11503,C537251,22620,4 TRADITIONAL SPINNING TOPS,-8,2010-12-06 10:45:00,1.25,NaN,United Kingdom,2010-12-06,-10.00
11504,C537251,21890,S/6 WOODEN SKITTLES IN COTTON BAG,-2,2010-12-06 10:45:00,2.95,NaN,United Kingdom,2010-12-06,-5.90
11505,C537251,22564,ALPHABET STENCIL CRAFT,-5,2010-12-06 10:45:00,1.25,NaN,United Kingdom,2010-12-06,-6.25
11506,C537251,21891,TRADITIONAL WOODEN SKIPPING ROPE,-3,2010-12-06 10:45:00,1.25,NaN,United Kingdom,2010-12-06,-3.75
...,...,...,...,...,...,...,...,...,...,...
492207,C578097,22112,CHOCOLATE HOT WATER BOTTLE,-48,2011-11-22 17:31:00,4.25,NaN,United Kingdom,2011-11-22,-204.00
514984,C579757,47469,ASSORTED SHAPES PHOTO CLIP SILVER,-24,2011-11-30 14:56:00,0.65,NaN,United Kingdom,2011-11-30,-15.60
516454,C579907,22169,FAMILY ALBUM WHITE PICTURE FRAME,-2,2011-12-01 08:48:00,7.65,NaN,EIRE,2011-12-01,-15.30
524601,C580604,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:35:00,11586.50,NaN,United Kingdom,2011-12-05,-11586.50


In [12]:
df[(df['CustomerID'].isna())&(df['InvoiceNo'].str.contains('[a-zA-Z]', regex=True))].describe(include='all')

<ipython-input-12-8f0fd4781de8>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df[(df['CustomerID'].isna())&(df['InvoiceNo'].str.contains('[a-zA-Z]', regex=True))].describe(include='all')


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
count,382,382,382,382.000000,382,382.000000,0.0,382,382,382.000000
unique,185,162,162,NaN,185,NaN,NaN,4,83,NaN
top,C557663,M,Manual,NaN,2011-06-21 17:59:00,NaN,NaN,United Kingdom,2011-06-21,NaN
freq,37,69,69,NaN,37,NaN,NaN,323,39,NaN
first,NaN,NaN,NaN,NaN,2010-12-06 10:45:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-05 11:36:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,-6.602094,NaN,708.341675,NaN,NaN,NaN,-775.791414
std,NaN,NaN,NaN,15.293928,NaN,2593.435141,NaN,NaN,NaN,2574.202702
min,NaN,NaN,NaN,-144.000000,NaN,-11062.060000,NaN,NaN,NaN,-17836.460000
25%,NaN,NaN,NaN,-5.750000,NaN,2.100000,NaN,NaN,NaN,-205.305000


Все эти инвойсы с отрицательной стоимостью. В описании перечислены как некоторые товары,  так и какие-то штрафы (от Амазона). Уникальных описаний не так много, можно оценить, что там фигурирует.

In [13]:
(
    df[(df['CustomerID'].isna())&(df['InvoiceNo'].str.contains('[a-zA-Z]', regex=True))]['Description']
    .value_counts().to_frame()
).head(15)

,Description
Manual,69
SAMPLES,60
AMAZON FEE,32
POSTAGE,29
Bank Charges,25
Adjust bad debt,3
PACK OF 72 RETROSPOT CAKE CASES,3
20 DOLLY PEGS RETROSPOT,2
BINGO SET,2
ASSORTED COLOUR BIRD ORNAMENT,2


Видим ручные правки, образцы, оплата доставки от амазона, почтовые расходы, банковские сборы и какие-то долги.

In [14]:
df[df['Description']=='Manual'].describe(include='all')

<ipython-input-14-a63fec0913c0>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df[df['Description']=='Manual'].describe(include='all')


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
count,567,567,567,567.000000,567,567.000000,460.000000,567,567,567.000000
unique,518,2,1,NaN,514,NaN,NaN,17,217,NaN
top,C548830,M,Manual,NaN,2011-04-04 13:03:00,NaN,NaN,United Kingdom,2011-07-18,NaN
freq,4,566,567,NaN,4,NaN,NaN,474,17,NaN
first,NaN,NaN,NaN,NaN,2010-12-01 15:35:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 10:28:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,5.158730,NaN,378.207160,15282.450000,NaN,NaN,-121.748924
std,NaN,NaN,NaN,120.732275,NaN,1779.373802,1882.606399,NaN,NaN,1841.235679
min,NaN,NaN,NaN,-1350.000000,NaN,0.000000,12352.000000,NaN,NaN,-38970.000000
25%,NaN,NaN,NaN,-1.000000,NaN,1.000000,13263.000000,NaN,NaN,-75.330000


StockCode всего 2 уникальных значения, посмотрим какие:

In [15]:
df[df['Description']=='Manual']['StockCode'].unique()

array(['M', 'm'], dtype=object)

Литеры "М" в различных видах написания. 

Сделаем предположение, что данные записи - какие-то ручные корректировки, которые стоит учесть в общей сумме, когда будет рассчитываться общий доход по странам и клиентам. Но если бы была такая возможность, лучше уточнить, что конкретно означают эти записи.

Изучим образцы:

In [16]:
df[df['Description']=='SAMPLES'].describe(include='all')

<ipython-input-16-4e72ed05e1e7>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df[df['Description']=='SAMPLES'].describe(include='all')


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
count,62,62,62,62.000000,62,62.000000,0.0,62,62,62.000000
unique,19,1,1,NaN,19,NaN,NaN,1,13,NaN
top,C544580,S,SAMPLES,NaN,2011-02-21 14:25:00,NaN,NaN,United Kingdom,2011-02-21,NaN
freq,15,62,62,NaN,15,NaN,NaN,62,26,NaN
first,NaN,NaN,NaN,NaN,2010-12-07 12:03:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-11-18 14:16:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,-0.935484,NaN,51.060484,NaN,NaN,NaN,-49.026613
std,NaN,NaN,NaN,0.356254,NaN,86.198654,NaN,NaN,NaN,87.390557
min,NaN,NaN,NaN,-1.000000,NaN,2.800000,NaN,NaN,NaN,-570.000000
25%,NaN,NaN,NaN,-1.000000,NaN,7.952500,NaN,NaN,NaN,-63.137500


Образцы почти все входят в разряд убытков, кроме 2 записей. Скорее всего, это какие-то единичные платные образцы, в большинстве случаев их поставляют в убыток, можно их оставить.

In [17]:
display(df[df['Description']=='AMAZON FEE'].describe(include='all'))
df[df['Description']=='AMAZON FEE']


<ipython-input-17-291999ef16c2>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display(df[df['Description']=='AMAZON FEE'].describe(include='all'))


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
count,34,34,34,34.000000,34,34.000000,0.0,34,34,34.000000
unique,34,1,1,NaN,34,NaN,NaN,1,15,NaN
top,C537651,AMAZONFEE,AMAZON FEE,NaN,2011-08-02 12:27:00,NaN,NaN,United Kingdom,2010-12-07,NaN
freq,1,34,34,NaN,1,NaN,NaN,34,7,NaN
first,NaN,NaN,NaN,NaN,2010-12-07 12:41:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-05 11:36:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,-0.882353,NaN,7324.784706,NaN,NaN,NaN,-6515.308824
std,NaN,NaN,NaN,0.477665,NaN,4619.549649,NaN,NaN,NaN,5734.365915
min,NaN,NaN,NaN,-1.000000,NaN,1.000000,NaN,NaN,NaN,-17836.460000
25%,NaN,NaN,NaN,-1.000000,NaN,5233.465000,NaN,NaN,NaN,-7322.685000


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
14514,C537600,AMAZONFEE,AMAZON FEE,-1,2010-12-07 12:41:00,1.00,NaN,United Kingdom,2010-12-07,-1.00
15016,C537630,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:04:00,13541.33,NaN,United Kingdom,2010-12-07,-13541.33
15017,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,2010-12-07,13541.33
16232,C537644,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:34:00,13474.79,NaN,United Kingdom,2010-12-07,-13474.79
16313,C537647,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:41:00,5519.25,NaN,United Kingdom,2010-12-07,-5519.25
16356,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541.33,NaN,United Kingdom,2010-12-07,-13541.33
16357,C537652,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:51:00,6706.71,NaN,United Kingdom,2010-12-07,-6706.71
43702,C540117,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:55:00,16888.02,NaN,United Kingdom,2011-01-05,-16888.02
43703,C540118,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:57:00,16453.71,NaN,United Kingdom,2011-01-05,-16453.71
96844,C544587,AMAZONFEE,AMAZON FEE,-1,2011-02-21 15:07:00,5575.28,NaN,United Kingdom,2011-02-21,-5575.28


Оставляем этот раздел.

In [18]:
df[(df['Description']=='CHRISTMAS TREE HANGING GOLD')&(df['CustomerID'].isna())]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
393624,570837,23222,CHRISTMAS TREE HANGING GOLD,12,2011-10-12 13:55:00,0.83,NaN,United Kingdom,2011-10-12,9.96
397665,C571196,23222,CHRISTMAS TREE HANGING GOLD,-12,2011-10-14 12:02:00,0.83,NaN,United Kingdom,2011-10-14,-9.96
430983,573585,23222,CHRISTMAS TREE HANGING GOLD,1,2011-10-31 14:41:00,1.63,NaN,United Kingdom,2011-10-31,1.63
437680,574298,23222,CHRISTMAS TREE HANGING GOLD,1,2011-11-03 15:56:00,1.63,NaN,United Kingdom,2011-11-03,1.63
450977,575177,23222,CHRISTMAS TREE HANGING GOLD,1,2011-11-08 18:41:00,1.63,NaN,United Kingdom,2011-11-08,1.63
457512,575739,23222,CHRISTMAS TREE HANGING GOLD,1,2011-11-11 09:05:00,1.63,NaN,United Kingdom,2011-11-11,1.63
459582,575875,23222,CHRISTMAS TREE HANGING GOLD,2,2011-11-11 13:06:00,1.63,NaN,United Kingdom,2011-11-11,3.26
477961,577078,23222,CHRISTMAS TREE HANGING GOLD,1,2011-11-17 15:17:00,1.63,NaN,United Kingdom,2011-11-17,1.63
496352,578344,23222,CHRISTMAS TREE HANGING GOLD,1,2011-11-24 09:21:00,1.63,NaN,United Kingdom,2011-11-24,1.63
507773,579187,23222,CHRISTMAS TREE HANGING GOLD,1,2011-11-28 15:31:00,1.63,NaN,United Kingdom,2011-11-28,1.63


По итогу я бы не стала удалять записи с отрицательными суммами, так как они отражают реальную картину:
- это отмены товаров;
- ручные корректировки (предположительно);
- оплата доставки Амазон;
- банковские сборы;
- бесплатные образцы.

Теперь проведем классификацию товаров с помощью машинного обучения.

In [19]:
from pymystem3 import Mystem

In [20]:
abc = 'One two and another numbers. I am counting right now.'
m = Mystem()

lemmas = m.lemmatize(abc)

In [21]:
lemmas

['One',
 ' ',
 'two',
 ' ',
 'and',
 ' ',
 'another',
 ' ',
 'numbers',
 '. ',
 'I',
 ' ',
 'am',
 ' ',
 'counting',
 ' ',
 'right',
 ' ',
 'now',
 '.',
 '\n']

In [22]:
from collections import Counter

In [23]:
Counter(lemmas)

Counter({'One': 1,
         ' ': 8,
         'two': 1,
         'and': 1,
         'another': 1,
         'numbers': 1,
         '. ': 1,
         'I': 1,
         'am': 1,
         'counting': 1,
         'right': 1,
         'now': 1,
         '.': 1,
         '\n': 1})

In [39]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()
# Lemmatize Single Word
print(lemmatizer.lemmatize("bats"))
#> bat
print(lemmatizer.lemmatize("are"))
#> are
print(lemmatizer.lemmatize("feet"))
#> foot

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Anastasia/nltk_data...


bat
are
foot


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011-12-09,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-09,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09,16.60


In [89]:
df['lemma'] = df['Description'].apply(
                                lambda x: ' '.join(
                                [lemmatizer.lemmatize(i.lower()) for i in re.findall(r'(?i)([a-zA-Z]+)', str(x))]))
df.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoce_day,total_amt,lemma
150957,549525,22856,ASSORTED EASTER DECORATIONS BELLS,1,2011-04-08 15:44:00,15.79,NaN,United Kingdom,2011-04-08,15.79,assorted easter decoration bell
522694,580432,22221,CAKE STAND LOVEBIRD 2 TIER PINK,12,2011-12-04 12:14:00,2.95,13153.0,United Kingdom,2011-12-04,35.40,cake stand lovebird tier pink
531868,580974,21934,SKULL SHOULDER BAG,10,2011-12-06 15:17:00,1.65,14293.0,United Kingdom,2011-12-06,16.50,skull shoulder bag
484132,577521,22469,HEART OF WICKER SMALL,2,2011-11-20 13:23:00,1.65,16411.0,United Kingdom,2011-11-20,3.30,heart of wicker small
121515,546769,22844,VINTAGE CREAM DOG FOOD CONTAINER,2,2011-03-16 14:57:00,8.50,17504.0,United Kingdom,2011-03-16,17.00,vintage cream dog food container
119302,546538,20972,PINK CREAM FELT CRAFT TRINKET BOX,12,2011-03-14 14:51:00,1.25,16327.0,United Kingdom,2011-03-14,15.00,pink cream felt craft trinket box
41143,539752,22221,CAKE STAND LOVEBIRD 2 TIER PINK,2,2010-12-21 15:43:00,9.95,12515.0,Italy,2010-12-21,19.90,cake stand lovebird tier pink
131400,547561,23127,DOLLCRAFT GIRL NICOLE,4,2011-03-24 08:44:00,4.95,14129.0,United Kingdom,2011-03-24,19.80,dollcraft girl nicole
424208,573245,82482,WOODEN PICTURE FRAME WHITE FINISH,1,2011-10-28 12:08:00,2.95,17841.0,United Kingdom,2011-10-28,2.95,wooden picture frame white finish
18317,537818,21844,RED RETROSPOT MUG,6,2010-12-08 13:53:00,2.95,12766.0,Portugal,2010-12-08,17.70,red retrospot mug


In [60]:
test = 'SET 7 BABUSHKA NESTING BOXES'
re.findall(r'(?i)([a-zA-Z]+)', test)

['SET', 'BABUSHKA', 'NESTING', 'BOXES']

In [59]:
text = 'ул. Карпинского, дом № 20, корпус 3, квартира 98'
re.findall(r'(?i)([а-я]+)', text)

['ул', 'Карпинского', 'дом', 'корпус', 'квартира']

In [40]:
test.split()

['SET', '7', 'BABUSHKA', 'NESTING', 'BOXES']

In [63]:
for i in re.findall(r'(?i)([a-zA-Z]+)', test):
    print(lemmatizer.lemmatize(i.lower()))

set
babushka
nesting
box


In [46]:
lemmatizer.lemmatize("BOXES")

'BOXES'

In [53]:
re.split(r'[ ]', test)

['SET', '7', 'BABUSHKA', 'NESTING', 'BOXES']

In [65]:
' '.join([lemmatizer.lemmatize(i.lower()) for i in re.findall(r'(?i)([a-zA-Z]+)', test)])

'set babushka nesting box'

In [69]:
df['Description'].unique()

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

In [76]:
def try_except_lambda(value, change):
    try:
        return value
    except:
        return change

In [85]:
for x in df['Description'].unique():
    try:
        ' '.join([lemmatizer.lemmatize(i.lower()) for i in re.findall(r'(?i)([a-zA-Z]+)', str(x))])
    except Exception as e: 
        print(e)
        print(x)

In [81]:
l = lambda x: ' '.join(
    [lemmatizer.lemmatize(i.lower()) for i in re.findall(r'(?i)([a-zA-Z]+)', str(x))])